In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Kernel abst
class Kernel:
    def __init__(self):
        return
    
    def computeKernelVec(self, x, x_all):
        return np.array([self.__call__(x, xi) for xi in x_all])
    
    def computeKernelMat(self, x_all):
        n = len(x_all)
        K = np.empty((n, n))
        for i, xi in enumerate(x_all):
            K[i,:] = self.computeKernelVec(xi, x_all)
        return K

# Kernel impl example
class IdentityKernel:
    def __init__(self):
        super().__init__()
        return
    
    def __call__(self, x, xi):
        return x
    
# Gauss Kernel
class GaussKernel(Kernel):
    def __init__(self, h=1.0):
        super().__init__()
        self.h = h
        return
    
    def __call__(self, x, xi):
        return np.exp(-np.linalg.norm(x-xi)**2/(2*self.h**2))
    
    def setParam(self, h):
        self.h = h
        return
     
# 
class LinearKernelModel:
    def __init__(self):
        return
    
    def __call__(self, kernel, x):
        return np.array([np.dot(self.param, kernel.computeKernelVec(xi, self.x_all)) for xi in x])
    
    def setParam(self, param, x_all):
        self.param = param
        self.x_all = x_all
        return

class L2SquareLoss:
    def __init__(self, l=1.0):
        self.l = l
        return
    
    def __call__(self, model, kernel, x, y):
        return (np.linalg.norm(model(kernel, x) - y)**2 + self.l * np.linalg.norm(model.param)**2)/2.0
    
    def setParam(self, l):
        self.l = l

# a whole model which has linear model, kernel, and criteria
class L2LinearKernelModel:
    def __init__(self):
        self.criteria = L2SquareLoss()
        self.model = LinearKernelModel()
        self.kernel = GaussKernel()
        return
    
    def __call__(self, x):
        return self.model(self.kernel, x)
        
    def train(self, x, y):
        K = self.kernel.computeKernelMat(x)
        U_inv = np.linalg.inv(np.dot(K.T, K) + self.criteria.l * np.eye(len(x)))
        self.model.setParam(np.dot(np.dot(U_inv, K.T), y), x)
        return
    
    def test(self, x, y):
        return self.criteria(self.model, self.kernel, x, y)
    
    def setParam(self, kernel_h, l2_lambda):
        self.kernel.setParam(kernel_h)
        self.criteria.setParam(l2_lambda)

In [ ]:
def cross_validation(x, y, k, model):
    n = len(x)
    if n % k != 0:
        print("haven't impl yet.")
        return
    
    interval = int(n / k)
    loss = 0
    for i in range(0,n,interval):
        train_x, train_y = np.concatenate([x[:i], x[i+interval:]], axis=0), np.concatenate([y[:i], y[i+interval:]], axis=0)
        valid_x, valid_y = x[i:i+interval], y[i:i+interval]
        model.train(train_x, train_y)
        loss += model.test(valid_x, valid_y)
        
    return loss / n

In [ ]:
# dataset
n = 50
x = np.linspace(-3,3,n)
y_true = np.sin(np.pi*x)/(np.pi*x) + 0.1 * x
y = y_true + 0.2 * np.random.randn(n)

In [ ]:
# test
model = L2LinearKernelModel()
model.train(x,y)
plt.plot(x,y, 'o')
plt.plot(x,y_true)
plt.plot(x,model(x))
plt.show()

In [ ]:
model = L2LinearKernelModel()
# candidates
h_list = [0.1, 1.0, 10.0]
l_list = [0.0001, 0.001, 0.01, 0.1]

# plot
fig,axes = plt.subplots(nrows=len(l_list),ncols=len(h_list),figsize=(15,12),sharex=True)
plt.subplots_adjust(wspace=0.4, hspace=0.6)

# for all candidates
for i,l in enumerate(l_list):
    for j,h in enumerate(h_list):
        model.setParam(h, l)
        loss = cross_validation(x,y,n/10,model)
        
        # draw
        axes[i,j].plot(x, y, 'o', linewidth=2)
        axes[i,j].plot(x, y_true, linewidth=2)
        axes[i,j].plot(x, model(x), linewidth=2, label=str(loss))
        axes[i,j].legend()
        axes[i,j].set_title("l="+str(l)+",h="+str(h))
        axes[i,j].grid(True)


# $\lambda = 0.001, h=1.0$が適切な正規化パラメータとガウス幅であると結論付けることができます。